In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
% matplotlib inline
import warnings
from tqdm import tqdm
from datetime import datetime
import json
warnings.filterwarnings("ignore")

In [3]:
os.chdir(r"C:\Users\Mohiuddin\Desktop\machine learning\ML\incedo-hackerearth")

In [4]:
train = pd.read_csv("train_file.csv")
tes = pd.read_csv("test_file.csv")
test = pd.read_csv("test_file.csv")
submission = pd.read_excel("sample_submission08f968d.xlsx")

In [6]:
train.head(2)

,Patient_ID,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,GeoLocation,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability
0,1,1995,"Houston, TX",1,Ever used cocaine,"any form of cocaine, such as powder, crack, or...",335.0,Female,Hispanic or Latino,4,"(29.760427, -95.369803)",H49,1,4,0,Local,7.4793
1,2,1997,"Houston, TX",0,Currently drank alcohol,"at least one drink of alcohol, on at least 1 d...",382.0,Total,Black or African American,4,"(29.760427, -95.369803)",H42,0,3,0,Local,49.6852


In [7]:
train.nunique()

Patient_ID                  55399
YEAR                           14
LocationDesc                   91
Subtopic                        2
Greater_Risk_Question          20
Description                    20
Sample_Size                  3583
Sex                             3
Race                            8
Grade                           5
GeoLocation                    85
QuestionCode                   20
StratID1                        3
StratID2                        8
StratID3                        5
StratificationType              5
Greater_Risk_Probability    51821
dtype: int64

In [8]:
train.isnull().sum()

Patient_ID                     0
YEAR                           0
LocationDesc                   0
Subtopic                       0
Greater_Risk_Question          0
Description                    0
Sample_Size                    0
Sex                            0
Race                           0
Grade                          0
GeoLocation                 3301
QuestionCode                   0
StratID1                       0
StratID2                       0
StratID3                       0
StratificationType             0
Greater_Risk_Probability       0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train["LocationDesc"] = labelencoder.fit_transform(train["LocationDesc"])
test["LocationDesc"] = labelencoder.fit_transform(test["LocationDesc"])


In [6]:
le = LabelEncoder()
train["Greater_Risk_Question"] = le.fit_transform(train["Greater_Risk_Question"])
test["Greater_Risk_Question"] = le.fit_transform(test["Greater_Risk_Question"])


In [7]:
te = LabelEncoder()
train["Description"] = te.fit_transform(train["Description"])
test["Description"] = te.fit_transform(test["Description"])


In [8]:
train.Sex.value_counts()

Total     22057
Female    16973
Male      16369
Name: Sex, dtype: int64

In [9]:
fe = LabelEncoder()
train["Sex"] = fe.fit_transform(train["Sex"])
test["Sex"] = fe.fit_transform(test["Sex"])


In [10]:
ge = LabelEncoder()
train["Race"] = ge.fit_transform(train["Race"])
test["Race"] = ge.fit_transform(test["Race"])


In [11]:
cc = LabelEncoder()
train["QuestionCode"] = cc.fit_transform(train["QuestionCode"])
test["QuestionCode"] = cc.fit_transform(test["QuestionCode"])


In [12]:
he = LabelEncoder()
train["StratificationType"] = he.fit_transform(train["StratificationType"])
test["StratificationType"] = he.fit_transform(test["StratificationType"])


In [13]:
#train["QuestionCode"] = train['QuestionCode'].replace('H', ' ', regex=True)

In [14]:
train['GeoLocation'] = train['GeoLocation'].str.strip('()')
test['GeoLocation'] = test['GeoLocation'].str.strip('()')


In [15]:
train[['Latitute','Longitute']] = train.GeoLocation.str.split(",",expand=True,).astype(float)
test[['Latitute','Longitute']] = test.GeoLocation.str.split(",",expand=True,).astype(float)

In [16]:
#train.Latitute.replace(np.nan,train["Latitute"].mean,inplace = True)
#test.Latitute.replace(np.nan,test["Latitute"].mean,inplace = True)

In [17]:
#train.Longitute.replace(np.nan,train["Longitute"].mean,inplace = True)
#test.Longitute.replace(np.nan,test["Longitute"].mean,inplace = True)

In [18]:
train['Latitute'].fillna((train['Latitute'].mean()), inplace=True)
test['Latitute'].fillna((test['Latitute'].mean()), inplace=True)
train['Longitute'].fillna((train['Longitute'].mean()), inplace=True)
test['Longitute'].fillna((test['Longitute'].mean()), inplace=True)


In [19]:
train.dtypes

Patient_ID                    int64
YEAR                          int64
LocationDesc                  int64
Subtopic                      int64
Greater_Risk_Question         int64
Description                   int64
Sample_Size                 float64
Sex                           int64
Race                          int64
Grade                         int64
GeoLocation                  object
QuestionCode                  int64
StratID1                      int64
StratID2                      int64
StratID3                      int64
StratificationType            int64
Greater_Risk_Probability    float64
Latitute                    float64
Longitute                   float64
dtype: object

In [20]:
train["stratIDTotal"] = train.StratID1 +  train.StratID2 + train.StratID3
test["stratIDTotal"] = test.StratID1 +  test.StratID2 + test.StratID3

In [21]:
y = train.Greater_Risk_Probability
y = pd.DataFrame(y)
train.drop(["GeoLocation","Patient_ID","Greater_Risk_Probability"],axis = 1, inplace = True)
test.drop(["GeoLocation","Patient_ID"],axis = 1, inplace = True)
print(train.shape)
print(test.shape)

(55399, 17)
(36932, 17)


In [146]:
train.head()

,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Latitute,Longitute,stratIDTotal
0,1995,32,1,6,7,335.0,0,3,4,9,1,4,0,0,29.760427,-95.369803,5
1,1997,32,0,0,8,382.0,2,2,4,2,0,3,0,0,29.760427,-95.369803,3
2,1993,54,0,14,14,196.0,0,7,0,1,1,6,2,3,39.493240,-117.071841,9
3,2017,71,1,6,7,104.0,1,1,4,9,2,2,0,3,40.793730,-77.860700,4
4,2007,77,1,5,15,100.0,0,4,4,15,1,7,0,0,37.774930,-122.419416,8


In [99]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=750)
model.fit(train, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [26]:
predictions = model.predict(test)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=       YEAR  LocationDesc  Subtopic  Greater_Risk_Question  Description  \
0      2007            52         1                     11            6   
1      2001            27         1                      1            5   
2      2005            14         1                     19           11   
3      2017            54         1                     13            0   
4      2007            34         1                      9            2   
5      2009            20         1                     19           11   
6      2009            75         1                      5           15   
7      2017            43         1                      3           18   
8      2011            27         1                      1            5   
9      2009            33         1                      7            1   
10     2017            89         1                     17            4   


In [119]:
pred = pd.DataFrame(pred)

In [121]:
pred.to_csv("RFS2",index = False)

In [145]:
final_result = pd.DataFrame({'Patient_ID':tes['Patient_ID'],'Greater_Risk_Probability':pred})
final_result = final_result[["Patient_ID","Greater_Risk_Probability"]]
final_result.to_csv('RfSolution2.csv',index=False)

TypeError: 'int' object is not iterable

In [109]:
import xgboost as xgb

In [110]:
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=8,
    learning_rate=0.01,
    n_estimators=1200,
    objective='reg:linear',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=27,
    **boost_params)

In [111]:
xgb0.fit(train, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='rmse', gamma=0,
       importance_type='gain', learning_rate=0.01, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=1200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=1)

In [28]:
predict_y = xgb0.predict(test)

NameError: name 'xgb0' is not defined

In [27]:
final_result = pd.DataFrame({'Patient_ID':tes['Patient_ID'],'Greater_Risk_Probability':predict_y})
final_result = final_result[["Patient_ID","Greater_Risk_Probability"]]
final_result.to_csv('xgbSolution3.csv',index=False)

NameError: name 'predict_y' is not defined

In [24]:
import lightgbm as lgb

d_train = lgb.Dataset(train, label=y)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
#params['objective'] = 'reg:linear'
params['metric'] = 'rmse'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

clf = lgb.train(params, d_train,1000)

In [28]:
predict_y = clf.predict(test)

In [29]:
final_result = pd.DataFrame({'Patient_ID':te['Patient_ID'],'Greater_Risk_Probability':predict_y})
final_result = final_result[["Patient_ID","Greater_Risk_Probability"]]
final_result.to_csv('lgbsubmission.csv',index=False)

In [43]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(train, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [44]:
predict_y = regressor.predict(test)

In [45]:
final_result = pd.DataFrame({'Patient_ID':te['Patient_ID'],'Greater_Risk_Probability':predict_y})
final_result = final_result[["Patient_ID","Greater_Risk_Probability"]]
final_result.to_csv('dtreesubmission.csv',index=False)

In [46]:
# Fitting Simple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(train, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [47]:
predict_y = regressor.predict(test)

In [48]:
final_result = pd.DataFrame({'Patient_ID':te['Patient_ID'],'Greater_Risk_Probability':predict_y})
final_result = final_result[["Patient_ID","Greater_Risk_Probability"]]
final_result.to_csv('linregsubmission.csv',index=False)

In [25]:
train.head()

,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Latitute,Longitute,stratIDTotal
0,1995,32,1,6,7,335.0,0,3,4,9,1,4,0,0,29.760427,-95.369803,5
1,1997,32,0,0,8,382.0,2,2,4,2,0,3,0,0,29.760427,-95.369803,3
2,1993,54,0,14,14,196.0,0,7,0,1,1,6,2,3,39.493240,-117.071841,9
3,2017,71,1,6,7,104.0,1,1,4,9,2,2,0,3,40.793730,-77.860700,4
4,2007,77,1,5,15,100.0,0,4,4,15,1,7,0,0,37.774930,-122.419416,8


In [23]:
#imports
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

#build our model
model = Sequential()

model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
model.fit(
    train,
    y,
    epochs=50,
    shuffle=True,
    verbose=2
)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=       YEAR  LocationDesc  Subtopic  Greater_Risk_Question  Description  \
0      1995            32         1                      6            7   
1      1997            32         0                      0            8   
2      1993            54         0                     14           14   
3      2017            71         1                      6            7   
4      2007            77         1                      5           15   
5      2015            72         1                     10           16   
6      2009            76         0                      0            8   
7      2007            85         0                      0            8   
8      2017            88         1                     19           11   
9      1999            31         1                     10           16   
10     2009            60         0                     14           14   
